In [1]:
import re
import unicodedata
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

# Dashbord de Profissionais de Dados

## Quem são? O que preferem? Quanto ganham? O que fazem?

Neste notebook feito em **Python** apresento vários insights que gerei a partir dos dados dos respondentes do **State of Data de 2021**.

O **objetivo** é entender mais detalhes sobre alguns dos **tipos de profissionais de dados** e como é a **evolução em relação aos níveis de carreira**.

As análises contém <u> apenas dados de profissionais CLT ou estágio com o intuito de deixar os dados mais comparáveis e por serem o maior percentual de dados</u>. Grupos como, por exemplo, profissionais que trabalham fora do país eram pouco representativos em termos de salário e quantidade de dados. Ademais, o salário de profissionais PJ é substancialmente maior.

Nos dados da pesquisa não havia a opção de estagiário em nível de carreira, mas havia uma coluna "('P2_a ', 'Qual sua situação atual de trabalho?')" que indicava se era estagiário. Nas análises exploratórias foi possível ver que havia dois picos diferentes nas distribuições dos juniores que não ocorria em plenos ou seniores. Ao fazer o deep dive, foi possível perceber que isso ocorria devido a mistura de juniores e estagiários. Dessa forma, para evitar ruídos e trazer mais clareza nas análises a seguir criei essa separação adicionando o dado de estagiário.

##### Profissionais analisados:
- Cientista de dados
- Engenheiro de dados
- Analista de dados

##### Níveis analisados:
- Estagiários
- Júnior
- Pleno
- Sênior

##### Tipos de contratos incluídos:
- CLT
- Estágio

In [2]:
def plot_gender(state_data_df):
    fig = plot_sunburst(
        state_data_df=state_data_df,
        column = "('P1_b ', 'Genero')"
    )
    return fig
    
def plot_work_model(state_data_df):
    fig = plot_sunburst(
        state_data_df = state_data_df,
        column = "('P2_r ', 'Qual a forma de trabalho ideal para você?')",
        replace_regex = "100% ",
        column_first=False
    )
    return fig

def plot_sunburst(state_data_df, column, replace_regex=None, column_first=True):
    df = state_data_df[["('P0', 'id')", column, "nível"]].copy()
    if replace_regex:
        df[column] = df[column].str.replace(r"\(.*\)","", regex=True)
        df[column] = df[column].str.replace("Modelo ","", regex=False)
        df[column] = df[column].str.replace(replace_regex, "", regex=True)
        df[column] = df[column].str.capitalize()
    grouped_df = df.groupby([column, "nível"]).count().reset_index()
    grouped_df.rename(columns={"('P0', 'id')": "contagem"}, inplace=True)
    path_list = [column, "nível"] if column_first else ["nível", column]
    fig = px.sunburst(grouped_df, path=path_list, values="contagem")
    fig.update_traces(textinfo="label+percent parent")
    return fig

def normalize_text(text):
    return unicodedata.normalize("NFKC", text)

def extract_salary(text):
    pattern = "R\$? ?(\d{1,2}\.?\d{3})\/mês?"
    match_list = re.findall(pattern,text)
    processed_match_list = [int(match.replace(".", "")) for match in match_list]
    return processed_match_list

def create_salary_str(salary_list):
    if len(salary_list) == 1:
        salary_value = salary_list[0]
        salary_str = f"Menos de R${salary_value}" if salary_value <= 1000 else f"Mais de R${salary_value}" 
    else:
        salary_str = f"R$ {'-'.join(map(str, salary_list))}"
    return salary_str

def get_salary_df(state_data_df, profession_name):
    salary_df = state_data_df[["('P0', 'id')", "('P2_h ', 'Faixa salarial')", "nível"]].copy()
    salary_df.dropna(axis="index", inplace=True)
    salary_df["salário_lista"] = salary_df["('P2_h ', 'Faixa salarial')"].map(normalize_text).map(extract_salary)
    salary_df["faixa_salarial"] = salary_df["salário_lista"].map(create_salary_str)
    salary_df["salário"] = salary_df["salário_lista"].map(lambda x: x[0])

    all_salary_df = pd.crosstab(
        [salary_df["faixa_salarial"], salary_df["salário"]],
        salary_df["nível"]
    ).reset_index()

    all_salary_df = all_salary_df.melt(
        id_vars=["faixa_salarial", "salário"], 
        var_name="nível", 
        value_name="contagem"
    )

    all_salary_df["porcentagem_cargo_nível_salário"] = all_salary_df["contagem"]/ all_salary_df.groupby("nível")["contagem"].transform('sum')
    all_salary_df["porcentagem_cargo_nível_salário"] *= 100
    all_salary_df = all_salary_df.round({"porcentagem_cargo_nível_salário": 2})
    all_salary_df.sort_values(by=["salário", "nível", "porcentagem_cargo_nível_salário"], ascending=True, inplace=True)
    return all_salary_df

def plot_salary(state_data_df, profession_name):
    salary_df = get_salary_df(state_data_df, profession_name)
    fig = px.bar(
        salary_df,
        x="faixa_salarial",
        y="porcentagem_cargo_nível_salário",
        color="nível",
        title=f"salário de um {profession_name}",
        custom_data=["contagem"]
    )
    return fig

def get_routine_column(profession_name):
    if profession_name == data_analyst_str:
        data_tec_column = "('P7_a ', 'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com análise de dados?')"
    elif profession_name == data_scientist_str:
        data_tec_column = "('P8_a ', 'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com ciência de dados?')"
    elif profession_name == data_engineer_str:
        data_tec_column = "('P6_a ', 'Quais das opções abaixo fazem parte da sua rotina no trabalho atual como engenheiro de dados?')"
    else:
        print("Valor inválido de profissão")
        data_tec_column = None
    return data_tec_column
    
def get_technology_routine_df(state_data_df, profession_name, data_tec_column):
    technology_routine_df = get_technology_df(
        state_data_df = state_data_df,
        data_tec_column = data_tec_column,
        profession_name = profession_name,
        tec_column_type = "routine"
    )
    return technology_routine_df

def plot_routine(state_data_df, profession_name):
    data_tec_column = get_routine_column(profession_name)
    technology_routine_df = get_technology_routine_df(state_data_df, profession_name, data_tec_column)
    technology_routine_df[data_tec_column] = technology_routine_df[data_tec_column].str[:100]
    technology_routine_df = technology_routine_df[technology_routine_df["porcentagem_cargo_nível_tecnologia"] >0]
    technology_routine_df.sort_values(by=["nível", "porcentagem_cargo_nível_tecnologia"], ascending=False, inplace=True)
    
    fig = px.icicle(
        technology_routine_df,
        path = [
            px.Constant(f"Rotina de um {profession_name}"),
            data_tec_column,
            "nível"
        ],
        values = "porcentagem_cargo_nível_tecnologia",
        color = "porcentagem_cargo_nível_tecnologia",
        #hover_data = ["total_cargo_nível_tecnologia"],
        color_continuous_scale = "GnBu",
    )
    
    fig.update_layout(
        margin = dict(t=50, l=25, r=25, b=25),
        uniformtext=dict(minsize=20)
    )
    fig.data[0]["textfont"]["size"] = 20
    return  fig

def filter_data_profession(state_data_df, profession):
    filtered_data = state_data_df[
        (state_data_df["cargo"] == profession) & (state_data_df["('P2_a ', 'Qual sua situação atual de trabalho?')"].isin(["Empregado (CLT)", "Estagiário"]))
    ].copy()
    return filtered_data

def plot_channels(plot_fig, subplot_fig, row, col):
    plot_fig.add_trace(
        subplot_fig["data"][0],
        row=row,
        col=col,
        exclude_empty_subplots=False
    )
    for ind in range(len(subplot_fig["data"])-1):
        plot_fig.append_trace(subplot_fig["data"][ind+1], row=row, col=col)

def create_layout_dict(profession_name):
    font = "PT Sans Narrow"
    color = "#515151"
    
    layout_dict = dict(
        width=1200,
        height=1400,
        paper_bgcolor="RGB(250, 250, 270)",
        margin=dict(l=100, r=100, t=100, b=30),
        autosize=False,
        hovermode="closest",
        title=dict(
            text = f"{profession_name}",
            x = 0.4,
            y = 0.98,
            xanchor = "center",
            yanchor = "top",
            font = dict(
                family = font,
                size = 30,
                color=color
            )
        ),
        legend_tracegroupgap = 10,
        font = dict(
            family = font,
            size = 14,
            color = "#515151"
        ),
        coloraxis1=dict(
            colorscale = "GnBu",
            colorbar_x = 1.0075,
            colorbar_thickness = 20
        ),
        coloraxis_colorbar = dict(
            yanchor = "top",
            x = 1.1,
            y = 0.2,
            ticks = "outside",
            ticksuffix = " %",
            len = 0.2
        ),
        legend = dict(
            font = dict(
                family = font,
                size = 10,
                color = color
            ),
            x = 1.1,
            y = 0.73
        ),
        yaxis3_range=[0, 100]
    )
    return layout_dict
        
def plot_dashboard(state_data_df, profession_name):
    fig1 = plot_gender(state_data_df)
    fig2 = plot_work_model(state_data_df)
    fig3 = plot_salary(state_data_df, profession_name)    
    fig4 = plot_data_source(
        technology_df = get_technology_df(
            state_data_df = state_data_df,
            data_tec_column = "('P4_b ', 'Quais das fontes de dados listadas você já analisou ou processou no trabalho?')",
            profession_name = profession_name,
            tec_column_type = "data_source"
        ),
        data_tec_column = "('P4_b ', 'Quais das fontes de dados listadas você já analisou ou processou no trabalho?')"
    )
    fig5 = plot_programming_language(state_data_df, profession_name)    
    fig6 = plot_routine(state_data_df, profession_name)

    fig = make_subplots(
        rows=4, cols=2,
        specs=[
            [{"type": "domain"}, {"type": "domain"}],
            [{"colspan": 2}, None],
            [{"type": "bar"}, {"type": "bar"}],
            [{"type": "domain", "colspan": 2}, None]
        ],
        shared_yaxes=False,
        start_cell="top-left",
        horizontal_spacing=0.065,
        vertical_spacing=0.08,
        row_heights=[0.45, 0.4, 0.5, 0.55],
        subplot_titles=(
            "Quem são?",
            "Qual modelo de trabalho preferem?",
            "Quanto ganham?",
            "Quais fontes de dados já usaram no trabalho?",
            "Quais linguagens usam no trabalho?",
            "O que fazem?",
        )
    )

    fig.add_trace(fig1["data"][0], row=1, col=1, exclude_empty_subplots=False)
    fig.add_trace(fig2["data"][0], row=1, col=2, exclude_empty_subplots=False)
    
    plot_channels(
        plot_fig=fig,
        subplot_fig=fig3,
        row=2,
        col=1
    )
    
    plot_channels(
        plot_fig=fig,
        subplot_fig=fig4,
        row=3,
        col=1
    )
    
    plot_channels(
        plot_fig=fig,
        subplot_fig=fig5,
        row=3,
        col=2
    )
    
    plot_channels(
        plot_fig=fig,
        subplot_fig=fig6,
        row=4,
        col=1
    )
    
    fig.update_layout(create_layout_dict(profession_name))
    fig.update_yaxes(title_text="Porcentagem de pessoas (%)", showgrid=True, row=2, col=1)
    fig.update_yaxes(title_text="Porcentagem de pessoas (%)", row=3, col=1)
    fig.update_annotations(font_size=14)
    fig.update_xaxes(tickangle=30)
    fig.show()
    
def get_technology_df(state_data_df, data_tec_column, profession_name, tec_column_type):
    tools_df = state_data_df[["('P0', 'id')", "cargo", "nível", data_tec_column]].copy()
    
    if tec_column_type == "routine":
        tools_df[data_tec_column] = tools_df[data_tec_column].str.replace(r".\)", r").", regex=True).str.split(pat=r"\.,\s")
        exploded_df = tools_df.explode(data_tec_column)
        exploded_df[data_tec_column] = exploded_df[data_tec_column].str.rstrip(".").str.replace(r"Sou responsável p((or)|(ela)) ", "", regex=True).str.capitalize().str.replace(r" \(.*\)","", regex=True)
    elif tec_column_type == "programming_language":
        tools_df[data_tec_column] = tools_df[data_tec_column].str.replace("Não utilizo nenhuma das linguagens listadas", "Nenhuma das opções", regex=False).str.split(pat=r",\s?",)
        exploded_df = tools_df.explode(data_tec_column)
    elif tec_column_type == "data_source":
        tools_df[data_tec_column] = tools_df[data_tec_column].str.split(pat=r",\s?",)
        exploded_df = tools_df.explode(data_tec_column)

    all_combinations_tec_df = pd.crosstab(
        [exploded_df[data_tec_column], exploded_df["cargo"]],
        exploded_df["nível"]
    ).reset_index()

    all_combinations_tec_df = all_combinations_tec_df.melt(
        id_vars=[data_tec_column, "cargo"], 
        var_name="nível", 
        value_name="total_cargo_nível_tecnologia"
    )
    
    data_count = state_data_df.groupby(["cargo", "nível"])["('P0', 'id')"].count().reset_index()
    data_count.rename(columns = {"('P0', 'id')":"total_cargo_nível"}, inplace = True)

    grouped_df = all_combinations_tec_df.merge(data_count, how="left", on=["nível", "cargo"])
    grouped_df["porcentagem_cargo_nível_tecnologia"] = round(100 * (grouped_df["total_cargo_nível_tecnologia"] / grouped_df["total_cargo_nível"]), 2)
    grouped_df.sort_values(by=["cargo", "nível", 'porcentagem_cargo_nível_tecnologia'], ascending=False, inplace=True)
    return grouped_df

def plot_programming_language(state_data_df, profession_name):
    technology_df = get_technology_df(
        state_data_df = state_data_df,
        data_tec_column = "('P4_d ', 'Quais das linguagens listadas abaixo você utiliza no trabalho?')",
        profession_name = profession_name,
        tec_column_type = "programming_language"
    )

    technology_df.rename(
        columns = {
            "('P4_d ', 'Quais das linguagens listadas abaixo você utiliza no trabalho?')": "linguagens_usadas"
        },
        inplace = True
    )

    technology_df["total_linguagem"] = technology_df.groupby(["linguagens_usadas"]).transform("sum")[["total_cargo_nível_tecnologia"]]

    technology_df = technology_df.sort_values(
        by=["total_linguagem", "nível", "porcentagem_cargo_nível_tecnologia", "linguagens_usadas"],
        ascending=[False, True, False, True],
    ).reset_index(drop=True)
    
    fig = px.bar(
        technology_df,
        x="linguagens_usadas",
        y="porcentagem_cargo_nível_tecnologia",
        color="nível",
        title=f"Linguagens usadas pelos {profession_name}",
        barmode="group",
        custom_data=["total_cargo_nível_tecnologia"]
    )
    fig.update_layout(yaxis={"categoryorder":"total ascending"})
    return fig

def plot_data_source(technology_df, data_tec_column):
    technology_df.rename(
        columns = {data_tec_column: "fontes_dados_usadas"},
        inplace = True
    )
    technology_df.sort_values(
        by=["nível", "porcentagem_cargo_nível_tecnologia", "fontes_dados_usadas"],
        ascending=[True, False, True],
        inplace=True
    )

    fig = px.line(
        technology_df,
        x = "nível",
        y = "porcentagem_cargo_nível_tecnologia",
        color = "fontes_dados_usadas",
        range_y = [0,100],
        title = "Evolução do uso do uso de fontes de dados por nível"
    )
    return fig

def read_state_data_df(path):
    state_data_df = pd.read_csv(path, low_memory=False)
    state_data_df.rename(
        columns = {
            "('P2_g ', 'Nivel')": "nível",
            "('P2_f ', 'Cargo Atual')": "cargo"
        },
        inplace = True
    )
    return state_data_df

def create_intern_column(state_of_data):
    intern_data_scientist_df = state_of_data.copy()
    intern_data_scientist_df.loc[
        (state_of_data["('P2_a ', 'Qual sua situação atual de trabalho?')"] == "Estagiário"),
        ["nível"]
    ] = "Estagiário"
    return intern_data_scientist_df

In [3]:
state_data_df = read_state_data_df("/kaggle/input/state-of-data-2021/State of Data 2021 - Dataset - Pgina1.csv")

In [4]:
data_scientist_str = "Cientista de Dados/Data Scientist"
data_analyst_str = "Analista de Dados/Data Analyst"
data_engineer_str = "Engenheiro de Dados/Data Engineer"

data_scientist_df = filter_data_profession(state_data_df, data_scientist_str)
data_scientist_df = create_intern_column(data_scientist_df)

data_analyst_df = filter_data_profession(state_data_df, data_analyst_str)
data_analyst_df = create_intern_column(data_analyst_df)

data_engineer_df = filter_data_profession(state_data_df, data_engineer_str)
data_engineer_df = create_intern_column(data_engineer_df)

In [5]:
plot_dashboard(data_scientist_df, data_scientist_str)

# Insights - Cientista de Dados:

---

### Quem são?
- **80%** dos cientistas de dados da pesquisa se identificaram como **homens**
- Apenas **20%** dos cientistas de dados se identificaram como **mulheres**
- **Nenhum** dos cientistas de dados se identificou como **outro gênero**

<div class="alert alert-block alert-warning">
<b>Insights:</b> Ainda há pouca diversidade de gênero entre os cientistas de dados do State of Data 2021. </div>

---

### Qual modelo de trabalho preferem?
- **61%** preferem trabalhar no modelo **híbrido flexível**
- **31%** preferem trabalhar no modelo **totalmente remoto**
- **6%** preferem trabalhar no modelo **híbrido com dias fixos de trabalho presencial**
- Apenas **2%** preferem o modelo **totalmente presencial**

<div class="alert alert-block alert-warning">
<b>Insights:</b> As preferências em relação ao modelo de trabalho são similares entre os cientistas de dados, independente do nível de carreira. A maioria dos cientistas de dados prefere o trabalho híbrido flexível. Quase um terço também prefere o trabalho totalmente remoto. Pouquíssimos preferem o trabalho completamente presencial. </div>

---

### Quanto ganham?
- Os cientistas de dados de nível estagiário ganhavam até **R\\$3.000** por mês, sendo que **61.29%** ganhavam de **R\\$1.000** a **R\\$2 mil**
- Aproximadamente **78.20%** dos cientistas de dados CLT de nível júnior ganhavam de **R\\$3.000** a **R\\$8.000** por mês, sendo a faixa salarial mais frequente de **R\\$4.000** a **R\\$6.000**
- Aproximadamente **73.07%** dos cientistas de dados CLT de nível pleno ganhavam de **R\\$6.000** a **R\\$12.000** por mês, sendo que as faixas salariais **R\\$6.000** a **R\\$8.000** e **R\\$8.000** a **R\\$12.000** tem percentual similar
- Aproximadamente **68%** dos cientistas de dados CLT de nível sênior ganhavam de **R\\$8.000** a **R\\$16.000** por mês, sendo que as faixas salariais **R\\$8.000** a **R\\$12.000** e **R\\$12.000** a **R\\$16.000** tem percentual similar

<div class="alert alert-block alert-success">
<b>Insights:</b> Há uma diferença significativa de salário com o aumento de senioridade. É possível perceber isso no gráfico "Quanto ganham?", pois quanto mais alto o nível de carreira mais deslocada para frente é a distribuição de salário. Os níveis plenos e seniores são os que tem maior interseção nas faixas salariais. Por exemplo, há uma probabilidade alta de um profissional pleno ou sênior ganhar de R\$8 mil a R\$12 mil, mas salários a partir de R\$12 mil são mais prováveis para profissionais de nível sênior. </div>

---

### Quais fontes de dados já usaram?
Já usaram **dados relacionais (estruturados em banco SQL)** no trabalho:
- **83.87%** dos cientistas de dados de nível **estagiário**
- **89.74%** dos cientistas de dados de nível **júnior**
- **98.46%** dos cientistas de dados de nível **pleno**
- **97.33%** dos cientistas de dados de nível **sênior**

Já usaram **planilhas** no trabalho:
- **83.87%** dos cientistas de dados de nível **estagiário**
- **88.46%** dos cientistas de dados de nível **júnior**
- **90.77%** dos cientistas de dados de nível **pleno**
- **82.67%** dos cientistas de dados de nível **sênior**

Já usaram **textos/documentos** no trabalho:
- **48.39%** dos cientistas de dados de nível **estagiário**
- **44.87%** dos cientistas de dados de nível **júnior** 
- **65.38%** dos cientistas de dados de nível **pleno**
- **76%** dos cientistas de dados de nível **sênior**

<div class="alert alert-block alert-info">
<b>Insights:</b> A fonte de dados mais usada por todos os níveis de carreira de cientista de dados foi dados relacionais (estruturados em banco SQL)
Similarmente, a segunda fonte de dados mais usada foi planilhas.
A terceira foi textos/documentos.
As fontes menos utilizadas foram dados estruturados não armazenados em nenhum banco, arquivos csv e dados de sensores.
Há uma correlação entre aumento de senioridade e aumento do uso de dados não estruturados (texto, áudio, imagens, ...). É interessante, pois esses tipos de dados muitas vezes exigem outros tipos de processamento e modelos de machine learning, podendo ser mais complexos que a manipulação de dados estruturados. Por exemplo, a manipulação de textos pode exigir tokenização, processamento de palavras para diminuição de vocabulários e outros tipos de visualização como word tree, nuvem de palavras, etc. </div>

---

### Quais linguagens mais usam?
Utilizam **Python** no trabalho:
- **83.87%** dos cientistas de dados de nível **estagiário** 
- **93.59%** dos cientistas de dados de nível **júnior** 
- **97.69%** dos cientistas de dados de nível **pleno**
- **100%** dos cientistas de dados de nível **sênior**

Utilizam **SQL** no trabalho:
- **77.42%** dos cientistas de dados de nível **estagiário** 
- **84.62%** dos cientistas de dados de nível **júnior** 
- **90.77%** dos cientistas de dados de nível **pleno**
- **88%** dos cientistas de dados de nível **sênior**

Utilizam **R** no trabalho:
- **25.81%** dos cientistas de dados de nível **estagiário** 
- **33.33%** dos cientistas de dados de nível **júnior**
- **31.54%** dos cientistas de dados de nível **pleno**
- **36%** dos cientistas de dados de nível **sênior**

<div class="alert alert-block alert-info">
<b>Insights:</b> Não há diferenças na ordem das 3 linguagens mais usadas no trabalho  de acordo com nível de carreira. Python é a linguagem mais popular entre os cientistas de dados no trabalho. Similarmente, SQL também é muito utilizado (reiterando também a informação das fontes de dados anterior). A linguagem R, que embora seja bem conhecida nos mundos de ciência de dados e estatística, aparece em terceiro lugar com uma frequência aproximadamente 3 vezes menor que Python. </div>

---

### O que fazem?

|                	| Desenvolvo modelos de machine learning com o objetivo de colocar em produção em sistemas 	| Estudos ad-hoc com o objetivo de confirmar hipóteses, realizar modelos preditivos, forecasts, análise de cluster para resolver problemas pontuais e responder perguntas das áreas de negócio 	| Coleta e limpeza dos dados que uso para análise e modelagem 	| Entrar em contato com os times de negócio para definição do problema, identificar a solução e apresentação de resultados 	| Cuido da manutenção de modelos de machine learning já em produção, atuando no monitoramento, ajustes e refatoração quando necessário 	|
|:--------------:	|:----------------------------------------------------------------------------------------:	|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:	|:-----------------------------------------------------------:	|--------------------------------------------------------------------------------------------------------------------------	|--------------------------------------------------------------------------------------------------------------------------------------	|
|   **Sênior**   	|                                          78.67%                                          	|                                                                                            70.67%                                                                                           	|                           65.33%                           	| 64%                                                                                                                     	| 41.33%                                                                                                                              	|
|    **Pleno**   	|                                          65.38%                                         	|                                                                                            71.54%                                                                                           	|                           60.77%                           	| 64.62%                                                                                                                  	| 32.31%                                                                                                                              	|
|   **Júnior**   	|                                          62.82%                                         	|                                                                                            56.41%                                                                                           	|                           55.13%                           	| 47.44%                                                                                                                  	| 24.36%                                                                                                                              	|
| **Estagiário** 	|                                          48.39%                                         	|                                                                                            32.26%                                                                                           	|                           32.26%                           	| **9.68%**                                                                                                                   	| **16.13%*


|                	| Colocar modelos em produção, criar os pipelines de dados, apis de consumo e monitoramento 	| Realizo construções de dashboards em ferramentas de bi como powerbi, tableau, looker, qlik, etc 	| Crio e dou manutenção em etls, dags e automações de pipelines de dados 	| Utilizo ferramentas avançadas de estatística como sas, spss, stata etc, para realizar análises 	| Crio e gerencio soluções de feature store e cultura de mlops 	| Criar e manter a infra que meus modelos e soluções rodam 	| Nenhuma das opções listadas refletem meu dia a dia 	|
|:--------------:	|:-----------------------------------------------------------------------------------------:	|:-----------------------------------------------------------------------------------------------:	|:----------------------------------------------------------------------:	|:----------------------------------------------------------------------------------------------:	|:------------------------------------------------------------:	|:--------------------------------------------------------:	|:--------------------------------------------------:	|
|   **Sênior**   	|                                          38.67%                                          	|                                              21.33%                                              	|                                  16%                                  	|                                             9.33%                                             	|                            5.33%                            	|                            8%                           	|                         0%                        	|
|    **Pleno**   	|                                          34.62%                                          	|                                             20.77%                                             	|                                  20%                                  	|                                             8.46%                                             	|                            8.46%                            	|                          6.92%                          	|                         0%                        	|
|   **Júnior**   	|                                          19.23%                                          	|                                             17.95%                                             	|                                 11.54%                                	|                                             3.23%                                             	|                            1.74%                            	|                          1.28%                          	|                         0%                        	|
| **Estagiário** 	|                                           12.9%                                          	|                                             16.13%                                             	|                                 6.45%                                 	|                                             2.56%                                             	|                            2.56%                            	|                            0%                           	|                       3.23%                       	|

<div class="alert alert-block alert-info">
<b>Insights:</b> 
Das 12 opções de tarefas de rotinas de cientista de dados, as 5 mais escolhidas são as que mais estão ligadas diretamente ao papel, pois envolvem desenvolvimento de modelos, estudos para responder perguntas do negócio, coleta e limpeza de dados, alinhamentos com time de negócio para definição do problema e manutenção do código de modelos em produção.
Dentre as tarefas não tão frequentes muitas fazem interseção com algumas das responsabilidades de engenheiro de machine learning e engenheiro de dados.
Os cientistas de dados seniores e plenos possuem uma variação maior de responsabilidades no dia a dia que juniores ou estagiários. Essa variação está presente principalmente em tarefas que fogem mais do escopo de atuação, as quais tem uma participação significativamente menor pelos profissionais menos experientes.
Em particular, os estagiários tem também uma grande diferença com os seniores em termos de participação de alinhamento com a área de negócio e manutenção de modelos. Essas informações são interessantes pois naturalmente um estagiário atua mais no operacional que um sênior e frequentemente não tem experiência com manutenção e cuidados de deterioramento de modelos, pois o  foco é muito mais voltado ao aprendizado e desenvolvimento de passos básicos.</div>

In [6]:
plot_dashboard(data_engineer_df, data_engineer_str)

# Insights - Engenheiro de Dados:

---

### Quem são?
- **86%** dos engenheiros de dados da pesquisa se identificaram como **homens**
- Apenas **14%** dos engenheiros de dados se identificaram como **mulheres**
- **Nenhum** dos engenheiros de dados se identificou como **outro gênero**

<div class="alert alert-block alert-warning">
<b>Insights:</b> Há pouca diversidade de gênero entre os engenheiros de dados do State of Data 2021. A proporção de gênero é ainda mais desbalanceada que nos profissionais cientistas de dados. </div>

---

### Qual modelo de trabalho preferem?
Modelo de trabalho preferido pelos engenheiros de dados:


|                	| **Remoto** 	| **Híbrido flexível** 	| **Híbrido com dias fixos** 	| **Presencial** 	|
|:--------------:	|:----------:	|:--------------------:	|:--------------------------:	|:--------------:	|
| **Estagiário** 	|     26%    	|          65%         	|             6%             	|       3%       	|
|   **Júnior**   	|     31%    	|          59%         	|             9%             	|       1%       	|
|    **Pleno**   	|     33%    	|          60%         	|             6%             	|       1%       	|
|   **Sênior**   	|     56%    	|          31%         	|             0%             	|       2%       	|

<div class="alert alert-block alert-warning">
<b>Insights:</b> As preferências em relação ao modelo de trabalho são similares entre engenheiros de dados juniores, plenos e estagiários.
O modelo escolhido por estes níveis de carreira foi o híbrido flexível.
Porém se compararmos as preferências de um estagiário com um sênior podemos ver uma grande diferença percentual.
Pouco mais da metade dos seniores preferem o modelo de trabalho remoto e há uma menor preferência por modelos híbridos. Em todos os níveis de carreira o modelo totalmente presencial foi o menos escolhido.</div>

---

### Quanto ganham?
- Os engenheiros de dados de nível estagiário ganhavam de **R\\$1.000** a **R\\$4.000** por mês, sendo que **53.85%** ganhavam de **R\\$1.000** a **R\\$2.000**
- Aproximadamente **74.13%** dos engenheiros de dados CLT de nível júnior ganhavam de **R\\$3.000** a **R\\$6.000** por mês, sendo a faixa salarial mais frequente de **R\\$4.000** a **R\\$6.000**
- Aproximadamente **76.04%** dos engenheiros de dados CLT de nível pleno ganhavam de **R\\$6.000** a **R\\$12.000** por mês, sendo a faixa salarial mais frequente de **R\\$6.000** a **R\\$8.000**
- Aproximadamente **73.99%** dos engenheiros de dados CLT de nível sênior ganhavam de **R\\$8.000** a **R\\$16.000** por mês, sendo a faixa salarial mais frequente de **R\\$8.000** a **R\\$12.000**
<div class="alert alert-block alert-success">
<b>Insights:</b> Há uma diferença significativa de salário com o aumento do nível de senioridade. É possível perceber isso no gráfico "Quanto ganham?", pois quanto mais alto o nível de carreira mais deslocada para frente é a distribuição de salário. Algumas faixas salárias possuem grande interseção entre 2 níveis. Por exemplo, há uma probabilidade alta de um profissional pleno ou sênior ganhar de R\$8 mil a R\$12 mil por mês, mas salários a partir de R\$12 mil são mais prováveis para profissionais de nível sênior.</div>

---

### Quais fontes de dados já usaram?
Já usaram **dados relacionais (estruturados em banco SQL)** no trabalho:
- **100%** dos engenheiros de dados de nível **estagiário**
- **96.55%** dos engenheiros de dados de nível **júnior**
- **100%** dos engenheiros de dados de nível **pleno**
- **98.63%** dos engenheiros de dados de nível **sênior**

Já usaram **planilhas** no trabalho:
- **84.62%** dos engenheiros de dados de nível **estagiário**
- **86.21%** dos engenheiros de dados de nível **júnior**
- **91.67%** dos engenheiros de dados de nível **pleno**
- **82.19%** dos engenheiros de dados de nível **sênior**

Já usaram **dados armazenados em bancos NoSQL** no trabalho:
- **30.77%** dos engenheiros de dados de nível **estagiário**
- **70.69%** dos engenheiros de dados de nível **júnior** 
- **69.79%** dos engenheiros de dados de nível **pleno**
- **80.82%** dos engenheiros de dados de nível **sênior**

<div class="alert alert-block alert-info">
<b>Insights:</b> A fonte de dados mais usada por todos os níveis de carreira de engenheiros de dados foi dados relacionais (estruturados em banco SQL). Similarmente, a segunda fonte de dados mais usada por esse profissional foi planilhas.
A terceira, dados armazenados em bancos NoSQL, sendo esta é mais frequente em profissionais com maior senioridade.</div>

---

### Quais linguagens mais usam?
Utilizam **SQL** no trabalho:
- **100%** dos engenheiros de dados de nível **estagiário** 
- **89.66%** dos engenheiros de dados de nível **júnior** 
- **97.92%** dos engenheiros de dados de nível **pleno**
- **97.26%** dos engenheiros de dados de nível **sênior**

Utilizam **Python** no trabalho:
- **76.92%** dos engenheiros de dados de nível **estagiário** 
- **91.38%** dos engenheiros de dados de nível **júnior** 
- **96.88%** dos engenheiros de dados de nível **pleno**
- **95.89%** dos engenheiros de dados de nível **sênior**

Utilizam **Scala** no trabalho:
- **0%** dos engenheiros de dados de nível **estagiário** 
- **18.97%** dos engenheiros de dados de nível **júnior**
- **12.5%** dos engenheiros de dados de nível **pleno**
- **20.55%** dos engenheiros de dados de nível **sênior**

<div class="alert alert-block alert-info">
<b>Insights:</b> SQL é a linguagem mais utilizada pelos engenheiros de dados no trabalho. Similarmente, a linguagem de programação Python também aparece com uma alta frequência de adoção entre os profissionais. Já Scala, que embora seja a linguagem nativa até para frameworks como Apache Spark e bem usadas nas grandes empresas de tecnologia como Netflix e Airbnb, aparece em terceiro lugar com uma frequência aproximadamente 5 vezes menor que SQL. Também é interessante perceber que há um aumento de adoção dessa linguagem pelos profissionais conforme ganham senioridade. A frequência de adoção de Python e Scala pelos engenheiros de dados estagiários é bem menor que os outros níveis de carreira, mas em SQL é superior.</div>

---

### O que fazem?

|                	| Desenvolvo pipelines de dados utilizando linguagens de programação como python, scala, java etc 	| Crio consultas através da linguagem sql para exportar informações e compartilhar com as áreas de negócio 	| Modelo soluções de arquitetura de dados, criando componentes de ingestão de dados, transformação e recuperação da informação 	| Cuido da qualidade dos dados, metadados e dicionário de dados 	|
|:--------------:	|:-----------------------------------------------------------------------------------------------:	|:--------------------------------------------------------------------------------------------------------:	|:----------------------------------------------------------------------------------------------------------------------------:	|:-------------------------------------------------------------:	|
|   **Sênior**   	|                                             76.71%                                             	|                                                  54.79%                                                 	|                                                            69.86%                                                           	|                             58.9%                            	|
|    **Pleno**   	|                                             86.46%                                             	|                                                  67.71%                                                 	|                                                            69.79%                                                           	|                            47.92%                            	|
|   **Júnior**   	|                                             79.31%                                             	|                                                  70.69%                                                 	|                                                          **56.9%**                                                          	|                            53.45%                            	|
| **Estagiário** 	|                                             69.23%                                             	|                                                  69.23%                                                 	|                                                          **38.46%**                                                         	|                            **30.77%**                        	|


|                	| Atuo na modelagem dos dados, com o objetivo de criar conjuntos de dados como data warehouses, data marts etc 	| Realizo construções de etls em ferramentas como pentaho, talend, dataflow etc 	| Desenvolvo/cuido da manutenção de repositórios de dados baseados em streaming de eventos como data lakes e data lakehouses 	| Atuo na integração de diferentes fontes de dados através de plataformas proprietárias como stitch data, fivetran etc 	|
|:--------------:	|:------------------------------------------------------------------------------------------------------------:	|:-----------------------------------------------------------------------------:	|:--------------------------------------------------------------------------------------------------------------------------:	|:--------------------------------------------------------------------------------------------------------------------:	|
|   **Sênior**   	|                                                    43.84%                                                   	|                                    39.73%                                   	|                                                           47.95%                                                          	|                                                        8.22%                                                        	|
|    **Pleno**   	|                                                    60.42%                                                   	|                                    41.67%                                    	|                                                           39.66%                                                          	|                                                        28.12%                                                       	|
|   **Júnior**   	|                                                    46.55%                                                   	|                                     43.1%                                    	|                                                           39.58%                                                          	|                                                        13.79%                                                       	|
| **Estagiário** 	|                                                    38.46%                                                   	|                                    53.85%                                    	|                                                             0%                                                            	|                                                        7.69%                                                        	|

<div class="alert alert-block alert-info">
<b>Insights:</b> 
Dentre as 4 tarefas mais frequentes no dia a dia de trabalho dos engenheiros de dados, foram: desenvolvimento de pipelines de dados, criação de consultas para compartilhar com a área de negócios, modelagem de soluções de arquitetura de dados e manutenção da qualidade dos dados. A modelagem de soluções de arquitetura de dados, criando componentes de ingestão de dados, transformação e recuperação da informação é mais frequentes em profissionais com mais experiência. Não é tão comum também estagiários fazerem manutenção da qualidade dos dados.
Por fim, atividades mais específicas relacionadas a data warehouses e datalakes, integração com outras fontes de dados apareceram com frequências menores. </div>

In [7]:
plot_dashboard(data_analyst_df, data_analyst_str)

# Insights - Analista de Dados:

---

### Quem são?
- **86%** dos analista de dados da pesquisa se identificaram como **homens**
- Apenas **14%** dos analista de dados se identificaram como **mulheres**
- **Nenhum** dos analistas de dados se identificou como **outro gênero**

<div class="alert alert-block alert-warning">
<b>Insights:</b> Ainda há pouca diversidade de gênero entre os analista de dados do State of Data 2021. </div>

---

### Qual modelo de trabalho preferem?
Modelo de trabalho preferido pelos analista de dados:

|                	| **Remoto** 	| **Híbrido flexível** 	| **Híbrido com dias fixos** 	| **Presencial** 	|
|:--------------:	|:----------:	|:--------------------:	|:--------------------------:	|:--------------:	|
| **Estagiário** 	|     28%    	|          56%         	|             16%             	|       0%       	|
|   **Júnior**   	|     37%    	|          55%         	|             7%             	|       1%       	|
|    **Pleno**   	|     26%    	|          71%         	|             3%             	|       0%       	|
|   **Sênior**   	|     32%    	|          61%         	|             7%             	|       0%       	|

<div class="alert alert-block alert-warning">
<b>Insights:</b> O modelo de trabalho mais popular entre todos os níveis de carreira de analistas de dados foi o híbrido flexível. Em segundo lugar apareceu o modelo totalmente remoto. Em seguida, com uma frequência bem menor foi o híbrido com dias fixos de trabalho presencial. Por fim, com uma frequência quase nula, o modelo totalmente presencial. </div>

---

### Quanto ganham?
- Os analistas de dados de nível estagiário ganhavam até **R\\$3.000** por mês, sendo que **48%** ganhavam de **R\\$1.000** a **R\\$2.000**
- Aproximadamente **85.72%** dos engenheiros de dados CLT de nível júnior ganhavam de **R\\$2.000** a **R\\$6.000** por mês, sendo a faixa salarial mais frequente de **R\\$4.000** a **R\\$6.000**
- Aproximadamente **71.28%** dos engenheiros de dados CLT de nível pleno ganhavam de **R\\$4.000** a **R\\$8.000** por mês, sendo que as faixas salariais **R\\$4.000** a **R\\$6.000**  e **R\\$6.000** a **R\\$8.000** tem percentual similar
- Aproximadamente **76.48%** dos engenheiros de dados CLT de nível sênior ganhavam de **R\\$6.000** a **R\\$12.000** por mês, sendo a faixa salarial mais frequente de **R\\$8.000** a **R\\$12.000**

<div class="alert alert-block alert-success">
<b>Insights:</b> Há uma diferença significativa de salário com o aumento do nível de senioridade. É possível perceber isso no gráfico "Quanto ganham?", pois quanto mais alto o nível de carreira mais deslocada para frente é a distribuição de salário. Algumas faixas salárias possuem grande interseção entre 2 níveis. Por exemplo, há uma probabilidade alta de um profissional pleno ou sênior ganhar de R\$6 mil a R\$8 mil por mês, mas salários a partir de R\$8 mil são mais prováveis para profissionais de nível sênior.</div>

---

### Quais fontes de dados já usaram?
Já usaram **planilhas** no trabalho:
- **96%** dos engenheiros de dados de nível **estagiário**
- **94.05%** dos engenheiros de dados de nível **júnior**
- **92.55%** dos engenheiros de dados de nível **pleno**
- **96.47%** dos engenheiros de dados de nível **sênior**

Já usaram **dados relacionais (estruturados em banco SQL)** no trabalho:
- **76%** dos engenheiros de dados de nível **estagiário**
- **83.33%** dos engenheiros de dados de nível **júnior**
- **97.87%** dos engenheiros de dados de nível **pleno**
- **97.65%** dos engenheiros de dados de nível **sênior**

Já usaram **textos/documentos** no trabalho:
- **64%** dos engenheiros de dados de nível **estagiário**
- **51.19%** dos engenheiros de dados de nível **júnior** 
- **44.68%** dos engenheiros de dados de nível **pleno**
- **51.76%** dos engenheiros de dados de nível **sênior**

<div class="alert alert-block alert-info">
<b>Insights:</b> A fonte de dados mais usada pelos analistas de dados no trabalho foi planilhas.
A segunda fonte de dados mais usada foi dados relacionais (estruturados em banco SQL).
Em seguida, textos/documentos.
Depois, com frequências similares dados georeferenciados e dados armazenados em bancos NoSQL. Em ambos os casos há uma correlação positiva de aumento de frequência e aumento de senioridade.
    Fontes como API, API JSON, PDF e dados scraped tiveram uma frequência bem baixa.
</div>

---

### Quais linguagens mais usam?
Utilizam **SQL** no trabalho:
- **76%** dos engenheiros de dados de nível **estagiário** 
- **83.33%** dos engenheiros de dados de nível **júnior** 
- **93.62%** dos engenheiros de dados de nível **pleno**
- **91.76%** dos engenheiros de dados de nível **sênior**

Utilizam **Python** no trabalho:
- **84%** dos engenheiros de dados de nível **estagiário** 
- **73.81%** dos engenheiros de dados de nível **júnior** 
- **78.72%** dos engenheiros de dados de nível **pleno**
- **68.24%** dos engenheiros de dados de nível **sênior**

Utilizam **R** no trabalho:
- **20%** dos engenheiros de dados de nível **estagiário** 
- **13.1%** dos engenheiros de dados de nível **júnior**
- **20.21%** dos engenheiros de dados de nível **pleno**
- **20%** dos engenheiros de dados de nível **sênior**

<div class="alert alert-block alert-info">
<b>Insights:</b> SQL é a linguagem mais utilizada pelos analistas de dados no trabalho. Similarmente,  Python também é muito utilizado. A linguagem R, aparece em terceiro lugar com uma frequência bem menor que Python. Não há diferenças na ordem das 3 linguagens mais usadas no trabalho  de acordo com nível para juniores, plenos e seniores, mas no caso dos estagiários Python é mais utilizado que SQL. </div>

---

### O que fazem?

|                	| Realizo construções de dashboards em ferramentas de bi como powerbi, tableau, looker, qlik etc 	| Crio consultas através da linguagem sql para exportar informações e compartilhar com as áreas de negócio 	| Processo e analiso dados utilizando linguagens de programação como python, r etc 	| Desenvolvo/cuido da manutenção de planilhas para atender as áreas de negócio 	|
|:--------------:	|:----------------------------------------------------------------------------------------------:	|:--------------------------------------------------------------------------------------------------------:	|:--------------------------------------------------------------------------------:	|:----------------------------------------------------------------------------:	|
|   **Sênior**   	|                                             78.82%                                            	|                                                  72.94%                                                 	|                                      52.94%                                     	|                                    34.12%                                   	|
|    **Pleno**   	|                                             69.15%                                            	|                                                  67.02%                                                 	|                                      51.06%                                     	|                                    19.15%                                   	|
|   **Júnior**   	|                                             57.14%                                            	|                                                  54.76%                                                 	|                                      52.38%                                     	|                                    45.24%                                   	|
| **Estagiário** 	|                                              76%                                              	|                                                   40%                                                   	|                                       52%                                       	|                                     44%                                     	|


|                	| Realizo experimentos e estudos utilizando metodologias estatísticas como teste de hipótese, modelos de regressão etc 	| Utilizo APIs para extrair dados e complementar minhas análises 	| Atuo na modelagem dos dados, com o objetivo de criar conjuntos de dados como data warehouses, data marts etc 	| Desenvolvo/cuido da manutenção de etl's utilizando tecnologias como talend, pentaho, airflow, dataflow etc 	| Utilizo ferramentas avançadas de estatística como sas, spss, stata etc, para realizar análises de dados 	| Nenhuma das opções listadas refletem meu dia a dia 	|
|:--------------:	|:--------------------------------------------------------------------------------------------------------------------:	|:--------------------------------------------------------------:	|:------------------------------------------------------------------------------------------------------------:	|:----------------------------------------------------------------------------------------------------------:	|:-------------------------------------------------------------------------------------------------------:	|:--------------------------------------------------:	|
|   **Sênior**   	|                                                        37.65%                                                       	|                             22.35%                            	|                                                    18.82%                                                   	|                                                   21.18%                                                  	|                                                  4.71%                                                 	|                         0%                        	|
|    **Pleno**   	|                                                        32.98%                                                       	|                             24.47%                            	|                                                    22.34%                                                   	|                                                   21.28%                                                  	|                                                  5.32%                                                 	|                         0%                        	|
|   **Júnior**   	|                                                        21.43%                                                       	|                             15.48%                            	|                                                    13.1%                                                    	|                                                   11.9%                                                   	|                                                  4.76%                                                 	|                       2.38%                       	|
| **Estagiário** 	|                                                         16%                                                         	|                              24%                              	|                                                     12%                                                     	|                                                     8%                                                    	|                                                   0%                                                   	|                         0%                        	|

<div class="alert alert-block alert-info">
<b>Insights:</b> 
    Dentre as 5 tarefas mais comuns do cotidiano de um analista de dados estão: construções de dashboards, criação de consultas para compartilhar informações com a área de negócio, processamento e análise de dados, desenvolvimento ou manutenção de planilhas que respondam perguntas do negócio e criação de experimentos ou estudos utilizando metodologias estatísticas. É interessante perceber que embora estas duas últimas tarefas citadas tenham tido frequência similar de forma geral entre analistas de dados, elas tem frequências bem diferentes em relação aos níveis de carreira. Por exemplo, a criação de experimentos ou estudos utilizando metodologias estatísticas vai aumentando a frequência de acordo com aumento de senioridade. Por outro lado, o desenvolvimento ou manutenção de planilhas que respondam perguntas do negócio é mais comum nos profissionais com menos experiência (juniores e estagiários).
Por fim, tarefas relacionadas a utilização de dados externos complementares, modelagem de dados, criação ou manutenção de ETLs e uso de ferramentas avançadas de estatística não foram tão frequentes. 
</div>